## Day 4!

In [13]:
import numpy as np
with open('data/day_4.txt') as f:
    lines = f.read().splitlines()

In [26]:
# read in the bingo draws
draws = [int(x) for x in lines[0].split(',')]

In [45]:
# read in all the boards into an Nx5x5 numpy array

Nboards = int((len(lines)-1)/6)
boards = np.zeros([Nboards, 5, 5], dtype=int)

new_board = False
num_board = -1
num_line = 0
for line in lines[1:]:
    if len(line) == 0:
        new_board = True
        num_board += 1
        num_line = 0
    else: 
        new_board = False
    
    if not new_board:
        new_line = [int(x) for x in line.split()]
        boards[num_board, num_line, :] = new_line
        num_line += 1

In [184]:
pull = 0
pulled_numbers = []
matched_numbers = np.zeros(boards.shape, dtype=bool)

for sample in draws:
    matched_numbers = np.logical_or(matched_numbers, boards == sample)
    best_board, best_idx, best_dim = check_board(matched_numbers)
    pull += 1
    if best_board:
        break
winning_board = boards[best_board,:,:].squeeze()
if best_dim == 'row':
    winning_line = winning_board[best_idx[0],:]
else:
    winning_line = winning_board[:, best_idx[0]]
print(f"Winning board {best_board[0]}\n\t{best_dim} {best_idx}")
print(boards[best_board,:,:])
print(winning_line)

Winning board 88
	row [4]
[[[41  6 58 42 85]
  [75 65 50  0  7]
  [82 80 12  5 61]
  [19 48 21 87 47]
  [71 14 24  8 23]]]
[71 14 24  8 23]


In [186]:
print(f"score = {(boards[best_board,:,:] * ~matched_numbers[best_board,:,:]).sum() * sample}")

score = 10374


In [187]:
time_to_win = np.zeros((Nboards,))
winning_boards = np.zeros((Nboards, 5,5))
for iboard in range(Nboards):
    board = boards[iboard,:,:].squeeze()
    pull = 0
    matched_numbers = np.zeros(board.shape, dtype=bool)
    winner = False
    for sample in draws:
        if not winner:
            matched_numbers = np.logical_or(matched_numbers, board == sample)
            if np.any(matched_numbers.sum(axis=0) == 5) or np.any(matched_numbers.sum(axis=1) == 5):             
                time_to_win[iboard] = pull
                winner = True
                winning_boards[iboard,:,:] = matched_numbers
            pull += 1

In [188]:
last_winner = time_to_win.argmax()
print(f"score = {(boards[last_winner] * (1-winning_boards[last_winner])).sum() * draws[int(time_to_win.max())]}")

score = 24742.0


In [109]:
def check_board(matched_numbers):
    best_board : int = False
    best_idx: int = False
    best_dim : str = 'none'
    if np.any(matched_numbers.sum(axis=1) == 5):
        winner = True
        best_dim = 'column'
        (best_board, best_idx) = np.where(matched_numbers.sum(axis=1) == 5)
    elif np.any(matched_numbers.sum(axis=2) == 5):
        winner = True
        best_dim = 'row'
        (best_board, best_idx) = np.where(matched_numbers.sum(axis=2) == 5)
        
    return best_board, best_idx, best_dim